In [1]:
import gc
import pywt
import numpy as np
import tensorflow as tf
from collections import Counter
from tensorflow import keras
from matplotlib import pyplot as plt 
from scipy.stats import mode
from sklearn.cross_validation import train_test_split
from statsmodels.robust import mad

#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
#set_session(tf.Session(config=config))

In [2]:
def my_metric(y_true, y_pred):
    y_true, y_pred = tf.reshape(y_true, [-1]), tf.reshape(y_pred, [-1])
    er = y_true - y_pred
    mask_n, mask_p = (er<=0), (er>0)
    er_n, er_p = tf.boolean_mask(er, mask_n), tf.boolean_mask(er, mask_p)
    score_n = tf.exp(-tf.log(0.5)*er_n/5)
    score_p = tf.exp(tf.log(0.5)*er_p/20)
    score = tf.concat([score_n, score_p], 0)
    score = tf.reduce_mean(score)*100
    return score

def metric_numpy(y_true, y_pred):
    y_true, y_pred = y_true.ravel(), y_pred.ravel()
    er = y_true - y_pred
    mask_n, mask_p = (er<=0), (er>0)
    er_n, er_p = er[mask_n], er[mask_p]
    score_n = np.exp(-np.log(0.5)*er_n/5)
    score_p = np.exp(np.log(0.5)*er_p/20)
    score = np.concatenate([score_n, score_p], 0)
    score = np.mean(score)*100
    return score

def my_loss(y_true, y_pred):
    y_true, y_pred = tf.reshape(y_true, [-1]), tf.reshape(y_pred, [-1])
    diff = tf.abs(y_true - y_pred)
    loss = tf.pow(diff, 3)
    return tf.reduce_mean(loss)

class Metric_Callback(keras.callbacks.Callback):
    def __init__(self, train_data, valid_data, t_l, v_l):
        self.train_data = train_data
        self.valid_data = valid_data
        self.t_l, self.v_l = t_l, v_l
        
    def on_epoch_end(self, batch, logs={}):
        y_pred = self.model.predict(self.train_data[0]).ravel() * self.t_l
        y_true = self.train_data[1] * self.t_l
        print("Train score %.4f"%metric_numpy(y_true, y_pred))
        
        y_pred = self.model.predict(self.valid_data[0]).ravel() * self.v_l
        y_true = self.valid_data[1] * self.v_l
        print("Valid score %.4f"%metric_numpy(y_true, y_pred))

def wavelet_smooth(data, wavelet="db4", level=4, m=1, n=4, plot=False):
    coeff = pywt.wavedec(data,wavelet,mode='sym', level=level)

    for i in range(m, n+1):
        length = len(coeff[i])
        threshold = np.sqrt(2*np.log(length))
        coeff[i] = pywt.threshold(coeff[i], value=threshold, mode="soft")

    denoised_data = pywt.waverec(coeff,wavelet,mode="sym")
    if plot:
        fig = plt.figure(figsize=(16,9))
        ax = fig.add_subplot(111)
        ax.plot(data)
        ax.plot(denoised_data)
        plt.show()
    return denoised_data

In [3]:
data = np.load("Output/01_l.npz",mmap_mode="r")

In [4]:
features, labels = data["data"], data["label"]
features = np.expand_dims(features, 3)
indices = np.random.permutation(features.shape[0])
features, labels = features[indices], labels[indices]

In [5]:
labels = np.expand_dims(labels, 1)
labels = np.concatenate([labels,]*5, axis=-1)
labels = np.expand_dims(labels, 2)

In [6]:
gc.collect()
train_X, test_X, train_y, test_y = train_test_split(features, labels)

In [20]:
C1D = tf.layers.Conv1D
TD = keras.layers.TimeDistributed
def build_model(shape_1):
    input_x = keras.layers.Input(shape=shape_1, name="X")
    
    x = TD(C1D(filters=8, kernel_size=7, strides=1, activation=None, padding="same"))(input_x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(C1D(filters=8, kernel_size=7, strides=1, activation=None, padding="same"))(x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(tf.layers.MaxPooling1D(pool_size=4, strides=4))(x)
    x = TD(tf.layers.Dropout(0.5))(x)
    
    x = TD(C1D(filters=16, kernel_size=7, strides=1, activation=None, padding="same"))(x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(C1D(filters=16, kernel_size=7, strides=1, activation=None, padding="same"))(x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(tf.layers.MaxPooling1D(pool_size=4, strides=4))(x)
    x = TD(tf.layers.Dropout(0.5))(x)
    
    x = TD(C1D(filters=32, kernel_size=7, strides=1, activation=None, padding="same"))(x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(C1D(filters=32, kernel_size=7, strides=1, activation=None, padding="same"))(x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(tf.layers.MaxPooling1D(pool_size=4, strides=4))(x)
    x = TD(tf.layers.Dropout(0.5))(x)
    
    x = TD(C1D(filters=16, kernel_size=7, strides=1, activation=None, padding="same"))(x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(C1D(filters=16, kernel_size=7, strides=1, activation=None, padding="same"))(x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(tf.layers.MaxPooling1D(pool_size=4, strides=4))(x)
    x = TD(tf.layers.Dropout(0.5))(x)
    
    x = TD(C1D(filters=8, kernel_size=7, strides=1, activation=None, padding="same"))(x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(C1D(filters=8, kernel_size=7, strides=1, activation=None, padding="same"))(x)
    #x = TD(tf.layers.BatchNormalization(axis=-1))(x)
    x = keras.layers.Activation(activation=tf.nn.relu)(x)
    x = TD(tf.layers.MaxPooling1D(pool_size=4, strides=4))(x)
    x = TD(tf.layers.Dropout(0.5))(x)
    
    x= keras.layers.Reshape((5, 200))(x)
    x= TD(keras.layers.Dense(512, activation=tf.nn.relu))(x)
    x= TD(keras.layers.Dense(256, activation=tf.nn.relu))(x)
    x = keras.layers.LSTM(256, return_sequences=True)(x)
    x = keras.layers.LSTM(256, return_sequences=True)(x)
    x = TD(keras.layers.Dense(256, activation=tf.nn.relu))(x)
    x = TD(keras.layers.Dense(128, activation=tf.nn.relu))(x)
    x = TD(keras.layers.Dense(64, activation=tf.nn.relu))(x)
    x = TD(keras.layers.Dense(16, activation=tf.nn.relu))(x)
    y = TD(keras.layers.Dense(1, activation=None))(x)
    
    model = keras.models.Model(inputs=input_x, outputs=y, name="base")
    return model

In [21]:
model = build_model((5,25600,1))

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
X (InputLayer)               (None, 5, 25600, 1)       0         
_________________________________________________________________
time_distributed_74 (TimeDis (None, 5, 25600, 8)       64        
_________________________________________________________________
activation_20 (Activation)   (None, 5, 25600, 8)       0         
_________________________________________________________________
time_distributed_75 (TimeDis (None, 5, 25600, 8)       456       
_________________________________________________________________
activation_21 (Activation)   (None, 5, 25600, 8)       0         
_________________________________________________________________
time_distributed_76 (TimeDis (None, 5, 6400, 8)        0         
_________________________________________________________________
time_distributed_77 (TimeDis (None, 5, 6400, 8)        0         
__________

In [26]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
losses = tf.losses.mean_squared_error
#losses = my_loss
model.compile(optimizer=optimizer, loss=losses, metrics=[my_metric])

In [27]:
#my_callback = Metric_Callback([[train_X_1, train_X_2], train_y], 
#                              [[test_X_1, test_X_2], test_y, 
#                              train_y[:, 1], test_y[:, 1])

In [ ]:
history = model.fit(train_X, train_y, epochs=256, batch_size=64, 
                    validation_data=[test_X, test_y])

Train on 1180 samples, validate on 394 samples
Epoch 1/256
1180/1180 [==============================] - 8s 7ms/step - loss: 4330.4782 - my_metric: 17.0969 - val_loss: 4423.2236 - val_my_metric: 15.8137
Epoch 2/256
1180/1180 [==============================] - 4s 4ms/step - loss: 4346.4791 - my_metric: 16.5422 - val_loss: 4367.8261 - val_my_metric: 16.2374
Epoch 3/256
1180/1180 [==============================] - 4s 4ms/step - loss: 4278.0735 - my_metric: 16.5259 - val_loss: 4376.3590 - val_my_metric: 16.0822
Epoch 4/256
 192/1180 [===>..........................] - ETA: 3s - loss: 4424.7733 - my_metric: 18.9804

In [ ]:
preds = model.predict([test_X_1, test_X_2])
diff = test_y.ravel() - preds.ravel()
diff.shape

In [ ]:
np.where((np.abs(diff)>100)==True)

In [ ]:
plt.plot(test_X_1[1739]) 

In [ ]:
model.save_weights("xx.weights")

In [ ]:
test_set = np.load("Output/02_test_a.npz")
test_data = test_set["data"]
test_label = test_set["label"]

In [ ]:
test_X = np.zeros((len(test_data), 25600, 1))
for idx, elem in enumerate(test_data):
    test_X[idx, :, 0] = np.fft.fft(elem[:, 0])

In [ ]:
preds = model.predict(test_X)
x = list()
for i in range(10):
    x += [i]*60

In [ ]:
preds = preds.reshape(5, -1)
for pred in preds:
    plt.scatter(x, pred)
    data = pred.reshape(-1, 60)
    mean_val = data.mean(axis=-1)
    data = data//5
    mode_val = mode(data, axis=-1)[0]*5
    print(mean_val[-1], mode_val[-1])
    plt.plot(mean_val)
    plt.plot(mode_val)
    plt.show()

In [ ]:
res = (test_label[:, 0:1]/(1-preds))*preds-test_label[:,1:2]
res = res.reshape(5, 10, 60)
res = res[:, -1, :]

In [ ]:
temp = res//10*10

In [ ]:
temp

In [ ]:
modes = mode(temp, axis=-1)[0]

In [ ]:
for i in range(5):
    print(res[i][(temp[i]>=modes[i]) & (temp[i]<(modes[i]+10))].mean())

In [ ]:
(temp[i]>=modes[i]) & (temp[i]<(modes[i]+10))

In [ ]:
a = tf.Variable([1,2,3,4])
mask = a > 2
b = tf.boolean_mask(a,mask)
plus = b+1

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(plus)

In [ ]:
sess.run(a)

In [ ]:
data_0 = features[160*0:160*2:2, :, 0].ravel()
data_1 = features[160*0:160*2:2, :, 1].ravel()
data_2 = features[160*0:160*2:2, :, 2].ravel()

In [ ]:
fig = plt.figure(figsize=(32,9))
ax = fig.add_subplot(111)
ax.plot(data_0)
plt.show()

In [ ]:
?np.load

In [ ]:
tf.Session()